# [E-14] Recommendation System
"Exploration Node 14. MovieLens Recommendation System" / 2022. 03. 01 (Tue) 이형주

## Contents
---
- **1. Environment Setup & Data Pre-Processing**
- **2. Model Training**
- **3. Project Retrospective**

## Rubric 평가기준
---

|  평가문항  |  상세기준  |
|:---------|:---------|
|1. CSR matrix가 정상적으로 만들어졌다.|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.
|2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
|3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

## 1. Environment Setup & Data Pre-Processing

In [1]:
import os
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
del ratings['timestamp']

In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies['title'] = movies['title'].str.lower()

In [7]:
del movies['genre']
movies.head()

,movie_id,title
0,1,toy story (1995)
1,2,jumanji (1995)
2,3,grumpier old men (1995)
3,4,waiting to exhale (1995)
4,5,father of the bride part ii (1995)


In [8]:
# ratings 와 movies 데이터 프레임 결합
data = ratings.join(movies.set_index('movie_id'), on='movie_id')
data.head()

,user_id,movie_id,count,title
0,1,1193,5,one flew over the cuckoo's nest (1975)
1,1,661,3,james and the giant peach (1996)
2,1,914,3,my fair lady (1964)
3,1,3408,4,erin brockovich (2000)
4,1,2355,5,"bug's life, a (1998)"


In [9]:
# 좋아하는 영화의 id 리스트
my_favoite_id = [1, 2294, 1566, 588, 1907]

# id 리스트를 title 리스트로 변환해준다
my_favorite_movie = []
for mid in my_favoite_id:
    my_favorite_movie.append(list(movies[movies['movie_id']==mid]['title'])[0])
    
# '7777'이라는 user_id가 위 영화를 5회씩 시청했다고 가정하겠습니다.
my_watchlist = pd.DataFrame({'user_id': [7777]*5, 'movie_id': my_favoite_id, 'title': my_favorite_movie, 'count':[5]*5})

if not data.isin({'user_id':[7777]})['user_id'].any():
    data = data.append(my_watchlist)                         

data = data[['user_id', 'movie_id', 'title', 'count']]
data.tail()

,user_id,movie_id,title,count
0,7777,1,toy story (1995),5
1,7777,2294,antz (1998),5
2,7777,1566,hercules (1997),5
3,7777,588,aladdin (1992),5
4,7777,1907,mulan (1998),5


In [10]:
##CSR Matrix

# 고유한 유저, 영화를 찾기
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 영화 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [11]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['new_user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')
    
# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['new_movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,title,count,new_user_id,new_movie_id
0,1,1193,one flew over the cuckoo's nest (1975),5,0,0
1,1,661,james and the giant peach (1996),3,0,1
2,1,914,my fair lady (1964),3,0,2
3,1,3408,erin brockovich (2000),4,0,3
4,1,2355,"bug's life, a (1998)",5,0,4
...,...,...,...,...,...,...
0,7777,1,toy story (1995),5,6039,40
1,7777,2294,antz (1998),5,6039,30
2,7777,1566,hercules (1997),5,6039,32
3,7777,588,aladdin (1992),5,6039,33


In [12]:
print(user_to_idx[7777])

6039


In [13]:
# CSR Matrix 생성
from scipy.sparse import csr_matrix

num_user = data['new_user_id'].nunique()
num_movie = data['new_movie_id'].nunique()

# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
csr_data = csr_matrix((data['count'], (data.new_user_id, data.new_movie_id)), shape= (num_user, num_movie))
csr_data

# implicit 라이브러리에서 권장하고 있는 세팅
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

##AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보면, 
## 1,4를 늘릴수록 학습을 잘 하게 되지만 과적합의 우려가 있습니다.

#1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
#2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
#3. use_gpu : GPU를 사용할 것인지
#4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

AlternatingLeastSquares.__init__

<method-wrapper '__init__' of function object at 0x7f1b01ee1b80>

## 2. Model Setup & Training

In [14]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [15]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

In [16]:
# 결과 확인

me, aladin = user_to_idx[7777], movie_to_idx['aladdin (1992)']
me_vector, aladin_vector = als_model.user_factors[me], als_model.item_factors[aladin]

In [17]:
me_vector

array([-0.1075675 ,  0.11115906,  0.06306721, -0.10668637, -0.24830517,
       -0.5343781 , -0.07046518, -0.5124264 , -0.18689057, -0.05626683,
       -0.3537821 ,  0.10888826,  0.60310125, -0.1390932 , -0.28461477,
        0.52054965, -0.06749066, -0.78787756,  0.54709506,  0.28197992,
       -0.38522023,  0.2696682 ,  0.09714293, -0.03601011,  0.02215356,
        0.15539686, -0.1621903 , -0.49300584,  0.65817994, -0.25478473,
        0.7692607 , -0.34792274, -0.08603022, -0.19902715, -0.23261477,
        0.3713764 , -0.08003888, -0.1989131 , -0.19648573, -0.07749288,
        0.25893632, -0.08846211,  0.03860112,  0.2301064 , -0.05300338,
        0.5118286 , -0.4955762 ,  0.59107935, -0.02437045, -0.09145676,
        0.05273393,  0.8795394 ,  0.37371615,  0.11014906, -0.6081359 ,
        0.01510751,  0.3012005 ,  0.22493258, -0.45775974,  0.14314456,
       -0.14439076,  0.81509393, -0.3160448 ,  0.00980403, -0.3200448 ,
       -0.15655442, -0.15591422, -0.51401955, -0.31161654,  0.71

In [18]:
aladin_vector

array([ 1.00171976e-02,  1.30376508e-02, -6.32406794e-04,  2.17908863e-02,
        3.97223746e-03, -1.51540227e-02, -5.51991584e-03, -6.20333897e-03,
        1.71045179e-03,  1.73464138e-02, -9.83464532e-04, -1.07662920e-02,
        9.99644026e-03,  2.55750176e-02, -1.16232652e-02,  1.45774649e-03,
        1.11785773e-02, -1.62405968e-02,  4.66853473e-03,  4.06905403e-03,
        2.66692624e-03, -4.71600768e-04,  2.59442199e-02,  1.81171894e-02,
        3.32198688e-03,  7.54258921e-03,  8.48122686e-03, -7.37191504e-03,
        1.42354127e-02, -7.44585739e-03,  1.90215986e-02, -1.04571814e-02,
        3.64216301e-03,  2.60716379e-02,  1.07468041e-02,  1.17098279e-02,
        3.90135753e-03, -5.93794370e-03, -2.39140631e-04,  3.36708175e-03,
        3.71378437e-02,  4.02524183e-03, -2.32960610e-03,  1.67230424e-02,
        2.60548200e-02,  2.14161146e-02, -1.13983992e-02,  2.63461508e-02,
        1.99454091e-02,  8.41216836e-03,  5.70987863e-03,  3.35965417e-02,
        1.72263645e-02,  

In [19]:
# 내적
np.dot(me_vector, aladin_vector)

0.74676114

In [20]:
# 모델의 선호도 예측 (Toy Story)
toy = movie_to_idx['toy story (1995)']
toy_vector = als_model.item_factors[toy]
np.dot(me_vector, toy_vector)

0.7443979

In [21]:
# 모델의 선호도 예측 (Antz)
antz = movie_to_idx['antz (1998)']
antz_vector = als_model.item_factors[antz]
np.dot(me_vector, antz_vector)

0.616909

In [22]:
# 모델의 선호도 예측 (Grumpier Old Men)
gom = movie_to_idx['grumpier old men (1995)']
gom_vector = als_model.item_factors[gom]
np.dot(me_vector, gom_vector)

## 전혀 다른 주제의 영화는 매우 낮은 선호도를 보여주는 점이 흥미롭다.
## Comedy|Romance -> Grumpier Old Men

-0.049136516

In [23]:
# 비슷한 영화 추천

def get_similar_artist(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [24]:
get_similar_artist('toy story (1995)')

['toy story (1995)',
 'toy story 2 (1999)',
 'aladdin (1992)',
 "bug's life, a (1998)",
 'babe (1995)',
 'beauty and the beast (1991)',
 'lion king, the (1994)',
 'groundhog day (1993)',
 'story of xinghua, the (1993)',
 'walking dead, the (1995)']

In [25]:
get_similar_artist('grumpier old men (1995)')

['grumpier old men (1995)',
 'grumpy old men (1993)',
 'naked gun 33 1/3: the final insult (1994)',
 'milk money (1994)',
 'nine months (1995)',
 'two if by sea (1996)',
 'odd couple ii, the (1998)',
 'vegas vacation (1997)',
 'out to sea (1997)',
 "fathers' day (1997)"]

In [26]:
get_similar_artist('terminator 2: judgment day (1991)')

['terminator 2: judgment day (1991)',
 'matrix, the (1999)',
 'terminator, the (1984)',
 'total recall (1990)',
 'jurassic park (1993)',
 'men in black (1997)',
 'fugitive, the (1993)',
 "she's the one (1996)",
 'aliens (1986)',
 'braveheart (1995)']

In [27]:
# 본인이 선호할 만한 영화 추천
## 대부분 에니메이션 영화류로 알맞게 추천되었다.

user = user_to_idx[7777]

# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)

# 인덱스를 영화 제목으로 변환
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
recommend_movies = [idx_to_movie[i[0]] for i in movie_recommended]
recommend_movies

['lion king, the (1994)',
 'beauty and the beast (1991)',
 "bug's life, a (1998)",
 'tarzan (1999)',
 'toy story 2 (1999)',
 'hunchback of notre dame, the (1996)',
 'anastasia (1997)',
 'iron giant, the (1999)',
 'little mermaid, the (1989)',
 'pocahontas (1995)',
 'prince of egypt, the (1998)',
 'james and the giant peach (1996)',
 'babe (1995)',
 '101 dalmatians (1961)',
 'cinderella (1950)',
 'nightmare before christmas, the (1993)',
 'aladdin and the king of thieves (1996)',
 'rescuers down under, the (1990)',
 'bambi (1942)',
 'secret of nimh, the (1982)']

In [28]:
## 기여도 (toy story 2 (1999))
toy2 = movie_to_idx['toy story 2 (1999)']
explain_toy2 = als_model.explain(user, csr_data, itemid=toy2)

# 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수)
[(idx_to_movie[i[0]], i[1]) for i in explain_toy2[1]]

[('toy story (1995)', 0.38877319032366),
 ('hercules (1997)', 0.06812616042324558),
 ('antz (1998)', 0.00045594498286977527),
 ('mulan (1998)', -0.006928438334313695),
 ('aladdin (1992)', -0.02705756442632363)]

In [29]:
## 기여도 (cinderella (1950))
cinderella = movie_to_idx['cinderella (1950)']
explain_cinderella = als_model.explain(user, csr_data, itemid=cinderella)

# 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수)
[(idx_to_movie[i[0]], i[1]) for i in explain_cinderella[1]]

[('mulan (1998)', 0.037300047100222444),
 ('antz (1998)', 0.035524219416544774),
 ('aladdin (1992)', 0.035298055418080204),
 ('hercules (1997)', 0.03364782775425251),
 ('toy story (1995)', 0.0237703004806976)]

## 3. Project Retrospective

+ 거의 85만건에 달하는 데이터였으나, 적은 샘플 사이즈 및 단시간 학습으로도 높은 적중률을 보이는 점이 인상깊다.
+ 더 높은 추천시스템의 적용을 위해서는 타 사용자들의 움직임, 패턴, 화면 머뭄시간, 이탈률 등을 대입할 수 있을 것 같다.
+ 고객의 소리(VOC)를 자동으로 분류하여 정리할 수 있는 프로젝트를 꼭 해보고 싶고, 그 프로젝트를 마치면 고도화 과정 중에 실행 우선순위를 정하는 추천 시스템을 만들어보고 싶다.